# Traffic assignment SO (congestion) (min)

$$
P_k =
\frac{\exp(-\mu \beta_{\text{time}} C_k)}
{\sum_{j \in \mathcal{K}_{od}} \exp(-\mu \beta_{\text{time}} C_j)}
$$


k_paths=5  
mu=100.0  
beta_time=1.0  
max_iter=500  
tol=1e-6  

In [ ]:
import math
import numpy as np
import pandas as pd
import networkx as nx


# ----------------------------
# 1) Read & normalize inputs
# ----------------------------
def read_net_csv(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip()

    if ";" in df.columns:
        df = df.drop(columns=[";"])

    required = ["A", "B", "capacity", "t0_min", "bpr_B", "bpr_power"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"net.csv missing columns: {missing}. got: {df.columns.tolist()}")

    out = pd.DataFrame()
    out["u"] = df["A"].astype(int)
    out["v"] = df["B"].astype(int)
    out["cap"] = df["capacity"].astype(float)
    out["t0"] = df["t0_min"].astype(float)
    out["alpha"] = df["bpr_B"].astype(float)       # BPR alpha, usually 0.15
    out["beta"] = df["bpr_power"].astype(float)    # BPR beta, usually 4

    return out


def read_od_csv(path: str) -> pd.DataFrame:
    od = pd.read_csv(path)
    od.columns = od.columns.str.strip()

    col_map = {}
    for c in od.columns:
        lc = c.lower()
        if lc in ["o", "origin"]:
            col_map[c] = "origin"
        elif lc in ["d", "dest", "destination"]:
            col_map[c] = "destination"
        elif lc in ["ton", "demand", "trips", "q"]:
            col_map[c] = "demand"
    od = od.rename(columns=col_map)

    required = ["origin", "destination", "demand"]
    missing = [c for c in required if c not in od.columns]
    if missing:
        raise ValueError(f"od.csv missing columns: {missing}. got: {od.columns.tolist()}")

    od = od[required].copy()
    od["origin"] = od["origin"].astype(int)
    od["destination"] = od["destination"].astype(int)
    od["demand"] = od["demand"].astype(float)

    # remove invalid rows
    od = od[(od["demand"] > 0) & (od["origin"] != od["destination"])].reset_index(drop=True)
    return od

# ----------------------------
# 2) BPR (UE) & SO marginal cost (congestion externality only)
# ----------------------------
def bpr_time(t0, x, cap, alpha=0.15, beta=4.0):
    """UE travel time t(x)"""
    if cap <= 0:
        return float("inf")
    z = x / cap
    return t0 * (1.0 + alpha * (z ** beta))

def bpr_marginal_time(t0, x, cap, alpha=0.15, beta=4.0):
    """
    SO (congestion externality only) marginal cost:
      t(x) + x dt/dx = t0 * (1 + alpha*(1+beta)*(x/c)^beta)
    """
    if cap <= 0:
        return float("inf")
    z = x / cap
    return t0 * (1.0 + alpha * (1.0 + beta) * (z ** beta))

def logit_probs(utilities, mu=1.0):
    umax = max(utilities)
    exps = [math.exp(mu * (u - umax)) for u in utilities]
    s = sum(exps)
    if s <= 0:
        return [1.0 / len(utilities)] * len(utilities)
    return [e / s for e in exps]

# ----------------------------
# 3) K shortest paths
# ----------------------------
def k_shortest_edge_paths(G, origin, dest, k=10, weight="weight"):
    try:
        gen = nx.shortest_simple_paths(G, origin, dest, weight=weight)
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        return []

    paths = []
    for _ in range(k):
        try:
            node_path = next(gen)
        except StopIteration:
            break
        edge_path = list(zip(node_path[:-1], node_path[1:]))
        paths.append(edge_path)
    return paths

# ----------------------------
# 4) Logit-SUE via MSA (SO only)
# ----------------------------
def run_logit_sue_so_congestion(net: pd.DataFrame,
                                od: pd.DataFrame,
                                k_paths=10,
                                mu=1.0,
                                beta_time=1.0,
                                max_iter=60,
                                tol=1e-5):
    """
    Solve SO using marginal congestion cost only:
      link weight = t(x) + x dt/dx
    Loading is Logit over K shortest paths + MSA.
    """

    edges = list(zip(net["u"].tolist(), net["v"].tolist()))

    # flows
    x = {e: 0.0 for e in edges}

    # graph
    G = nx.DiGraph()
    for (u, v), t0 in zip(edges, net["t0"].tolist()):
        G.add_edge(u, v, weight=float(t0))

    def compute_edge_costs_so(x_dict):
        c = {}
        for i, (u, v) in enumerate(edges):
            row = net.iloc[i]
            c[(u, v)] = bpr_marginal_time(row["t0"], x_dict[(u, v)], row["cap"], row["alpha"], row["beta"])
        return c

    def rel_change(x_old, x_new):
        num = sum(abs(x_new[e] - x_old[e]) for e in edges)
        den = max(1.0, sum(abs(x_old[e]) for e in edges))
        return num / den

    for it in range(1, max_iter + 1):
        # 1) update SO marginal costs as weights
        edge_costs = compute_edge_costs_so(x)
        for (u, v), cost in edge_costs.items():
            G[u][v]["weight"] = float(cost)

        # 2) auxiliary flow via logit loading
        x_hat = {e: 0.0 for e in edges}

        for _, r in od.iterrows():
            o = int(r["origin"])
            d = int(r["destination"])
            q = float(r["demand"])
            if q <= 0:
                continue

            cand_paths = k_shortest_edge_paths(G, o, d, k=k_paths, weight="weight")
            if not cand_paths:
                continue

            # utility: U = - beta_time * sum(cost)
            utils = []
            for p in cand_paths:
                cost_p = sum(edge_costs[e] for e in p)
                utils.append(-beta_time * cost_p)

            probs = logit_probs(utils, mu=mu)

            for p, pk in zip(cand_paths, probs):
                fp = q * pk
                for e in p:
                    x_hat[e] += fp

        # 3) MSA update
        step = 1.0 / it
        x_new = {e: x[e] + step * (x_hat[e] - x[e]) for e in edges}

        gap = rel_change(x, x_new)
        x = x_new
        print(f"[SO] Iter {it:02d} | step={step:.4f} | rel_change={gap:.6e}")

        if gap < tol:
            break

    # output
    out = net.copy()
    out["flow"] = [x[(u, v)] for (u, v) in edges]

    # realized travel time (UE time) for interpretation/comparison
    out["time"] = [
        bpr_time(out.loc[i, "t0"], out.loc[i, "flow"], out.loc[i, "cap"], out.loc[i, "alpha"], out.loc[i, "beta"])
        for i in range(len(out))
    ]

    # marginal cost used in SO (congestion externality only)
    out["mc_time"] = [
        bpr_marginal_time(out.loc[i, "t0"], out.loc[i, "flow"], out.loc[i, "cap"], out.loc[i, "alpha"], out.loc[i, "beta"])
        for i in range(len(out))
    ]
    return out

# ----------------------------
# 5) main (SO only)
# ----------------------------
if __name__ == "__main__":
    net = read_net_csv("/Users/keito-shiraki/traffic_assignment/data/csv/SiouxFalls_net_Vff60_with_minutes.csv")
    od = read_od_csv("/Users/keito-shiraki/traffic_assignment/data/csv/SiouxFalls_od.csv")

    result_so = run_logit_sue_so_congestion(
        net=net,
        od=od,
        k_paths=5,
        mu=100.0,
        beta_time=1.0,
        max_iter=500,
        tol=1e-6
    )

    result_so.to_csv("assignment_result_SO_congestion_min2.csv", index=False)
    print("Saved: assignment_result_SO_congestion_min2.csv")
